# VeloceReduction -- Tutorial

This tutorial provides an example on how to reduce data of a given night YYMMDD.

In [1]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
from astropy.io import fits
from pathlib import Path

# VeloceReduction modules and function
from VeloceReduction import config
from VeloceReduction.utils import identify_calibration_and_science_runs, polynomial_function
from VeloceReduction.extraction import extract_orders, extract_initial_order_ranges_and_coeffs
from VeloceReduction.calibration import calibrate_wavelength

## Adjust Date and Directory

In [2]:
# Define the Date
config.date = '001122' # Following the format YYMMDD

# Define the Working Directory
# We will assume that CCD images are saved in this directory under observations/
config.working_directory = './'

## Identfiy Calibration and Science Runs

In [3]:
# Extract the Calibration and Science data from the night log
calibration_runs, science_runs = identify_calibration_and_science_runs(config.date, config.working_directory+'observations/')



Identifying calibration and science runs now

Found Log file ./observations//001122/001122-AAT-example.log



## Extract orders and save in initial FITS files with an extension per order.

In [4]:
# Extract initial order ranges and coefficients
initial_order_ranges, initial_order_coeffs = extract_initial_order_ranges_and_coeffs()

In [5]:
# Extract Master Flat
print('Extracting Master Flat')
master_flat, noise = extract_orders(
    ccd1_runs = calibration_runs['Flat_60.0'][:1],
    ccd2_runs = calibration_runs['Flat_1.0'][:1],
    ccd3_runs = calibration_runs['Flat_0.1'][:1],
    Flat = True
)

# Extract Master ThXe
print('Extracting Master ThXe')
master_thxe, noise = extract_orders(
    ccd1_runs = calibration_runs['FibTh_180.0'][:1],
    ccd2_runs = calibration_runs['FibTh_60.0'][:1],
    ccd3_runs = calibration_runs['FibTh_15.0'][:1]
)

# Extract Master LC
print('Extracting Master LC')
master_lc, noise = extract_orders(
    ccd1_runs = calibration_runs['SimLC'][-1:],
    ccd2_runs = calibration_runs['SimLC'][-1:],
    ccd3_runs = calibration_runs['SimLC'][-1:],
    LC = True,
    # tramline_debug = True
)

Extracting Master Flat
Extracting Master ThXe
Extracting Master LC


In [6]:
# Extract Science Objects and save them into FITS files under reduced_data/
for science_object in list(science_runs.keys()):
    print('Extracting '+science_object)
    try:
        science, science_noise = extract_orders(
            ccd1_runs = science_runs[science_object],
            ccd2_runs = science_runs[science_object],
            ccd3_runs = science_runs[science_object],
            Science=True
        )

        # Create a primary HDU and HDU list
        primary_hdu = fits.PrimaryHDU()
        hdul = fits.HDUList([primary_hdu])

        # Loop over your extension names and corresponding data arrays
        for ext_index, ext_name in enumerate(initial_order_coeffs):
            # Create an ImageHDU object for each extension

            # There is not ThXe available for CCD1's order 138.
            if ext_name != 'ccd_1_order_138':

                # Define the columns with appropriate formats
                col1_def = fits.Column(name='wave_vac',format='E', array=np.arange(len(science[ext_index,:]),dtype=float))
                col2_def = fits.Column(name='wave_air',format='E', array=np.arange(len(science[ext_index,:]),dtype=float))
                col3_def = fits.Column(name='science', format='E', array=science[ext_index,:])
                col4_def = fits.Column(name='science_noise',   format='E', array=science_noise[ext_index,:])
                col5_def = fits.Column(name='flat',    format='E', array=master_flat[ext_index,:])
                col6_def = fits.Column(name='thxe',    format='E', array=master_thxe[ext_index,:])
                col7_def = fits.Column(name='lc',      format='E', array=master_lc[ext_index,:])

                hdu = fits.BinTableHDU.from_columns([col1_def, col2_def, col3_def, col4_def, col5_def, col6_def, col7_def], name=science_object+'_'+ext_name.lower())

                # Append the HDU to the HDU list
                hdul.append(hdu)

        # Save to a new FITS file with an extension for each order
        Path(config.working_directory+'reduced_data/'+config.date+'/'+science_object).mkdir(parents=True, exist_ok=True)    
        hdul.writeto(config.working_directory+'reduced_data/'+config.date+'/'+science_object+'/veloce_spectra_'+science_object+'_'+config.date+'.fits', overwrite=True)

        print('Successfully extracted '+science_object)

    except:
        print('Failed to extract '+science_object)

Extracting HIP69673
Successfully extracted HIP69673


## Wavelength calibration

In [7]:
for science_object in list(science_runs.keys()):
    calibrate_wavelength(science_object, create_overview_pdf=True)

/Users/buder/git/VeloceReduction/VeloceReduction/calibration.py:89: RuntimeWarning: invalid value encountered in true_divide
  ax.plot(file[order].data['SCIENCE']/file[order].data['FLAT'], lw=1)
/Users/buder/git/VeloceReduction/VeloceReduction/calibration.py:102: RuntimeWarning: invalid value encountered in true_divide
  ax.plot(file[order].data['SCIENCE']/file[order].data['SCIENCE_NOISE'], lw=1)
/Users/buder/git/VeloceReduction/VeloceReduction/calibration.py:106: RuntimeWarning: invalid value encountered in true_divide
  ax.plot(file[order].data['THXE']/file[order].data['FLAT'], lw=1)
/Users/buder/git/VeloceReduction/VeloceReduction/calibration.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ax.plot(file[order].data['LC']/file[order].data['FLAT'], lw=0.5)
/Users/buder/git/VeloceReduction/VeloceReduction/calibration.py:118: RuntimeWarning: invalid value encountered in true_divide
  science = file[order].data['SCIENCE']/file[order].data['FLAT']
/Users/buder/git/Veloc